# Constant Bidder

## Environment

In [93]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

## Data Loader

In [85]:
data_path = os.path.abspath(os.pardir + '/Data/validation.csv')
df = pd.read_csv(data_path, na_values=['Na', 'null']).fillna(0)

### Data Exploration

In [41]:
df_clicks = df.loc[df["click"] == 1]
df_clicks.reset_index(drop=True, inplace=True)

print(df_clicks.mean())

feature = 'advertiser'
df_grouped_mean = df_clicks.groupby([feature]).mean()
df_grouped_sum = df_clicks.groupby([feature]).sum()

df_grouped_mean['eCPC'] = df_grouped_sum['payprice'] / (df_grouped_sum['click'])
df_grouped_mean['ctr'] = df_grouped_sum['click'] / df['advertiser'].value_counts()

#print(df_grouped['eCPC'], "\n")

df_clicks.mean()
#df_grouped_mean

df_overview = pd.DataFrame(df_grouped_mean, columns=['ctr', 'eCPC', 'payprice', 'bidprice'])
print(df_overview)

click            1.000000
weekday          2.891089
hour            14.509901
region         149.707921
city           155.163366
adexchange       1.886139
urlid            0.000000
slotwidth      562.663366
slotheight     170.148515
slotformat       0.371287
slotprice       34.267327
bidprice       273.925743
payprice       105.717822
advertiser    2785.272277
dtype: float64
                 ctr        eCPC    payprice    bidprice
advertiser                                              
1458        0.000786   86.632653   86.632653  300.000000
2259        0.000120  177.500000  177.500000  285.500000
2261        0.000221   84.666667   84.666667  288.333333
2821        0.000868  141.086957  141.086957  291.782609
2997        0.004210  116.769231  116.769231  277.000000
3358        0.000698  118.565217  118.565217  231.826087
3386        0.000494  104.392857  104.392857  300.000000
3427        0.000737   93.837838   93.837838  237.837838
3476        0.000283   99.818182   99.818182  250.1

### Incoming bids

target and feature variables

'target' : 'click',     # 'payprice', 'bidprice'   # select the target

'features': ['weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP', 'region', 'city', 'adexchange', 'domain', 
'url', 'urlid', 'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice', 'creative', 'keypage', 'advertiser', 'usertag'], # select the data features


In [92]:
budget = 6250
constant_bid = 11

# 10, 1.  

## Evaluation Stats_____________

bids_won = 0
earned_clicks = 0
ctr = 0                  # bids_won / earned_clicks
total_paid = 0
cpc = 0                  # cost per click



constant_bids = np.arange(7,20,1)




#print("starting budget:", budget, "\n")

for constant_bid in constant_bids:
    
    ## shuffle dataframe rows
    df = df.sample(frac=1).reset_index(drop=True)
    
    print("\n\nconstant_bid:", constant_bid, "\n")
    
    ## Evaluation Stats_____________
    budget = 6250

    bids_won = 0
    earned_clicks = 0
    ctr = 0                  # bids_won / earned_clicks
    total_paid = 0
    cpc = 0                  # cost per click

    for index, row in df.iterrows():

        if constant_bid > budget: # check if budget is sufficient for bidprice
            constant_bid = budget
            #print("constant bid reduced to:", constant_bid, ", total_paid:", total_paid, ", bids_won:", bids_won, ", earned clicks:", earned_clicks, "\n")

        if budget <= 0:
            print("-- break after auction #", index)
            break

        # WON BID ______________________________________________

        if constant_bid >= row['payprice']:     

            bids_won += 1                        # won the bid
            total_paid += row['payprice']        # add amount to total_paid   
            budget = budget - row['payprice']    # substract money from budget

            #if constant_bid == row['bidprice']:      
                #budget = budget - row['payprice']    # substract money from budget

            #elif constant_bid > row['bidprice']:
            #    budget = budget - row['bidprice']    # substract money from budget

            # CLICK = 1 ______________________________________________

            if row['click'] == 1:    # only reduce money from budget if ad has been clicked

                    earned_clicks += 1                   # earn the click
                    #print("current budget:", budget, ", earned clicks:", earned_clicks, "\n")

        #if index%100000 == 0:
        #    print("bid#", index, ", budget:", budget, ", payprice:", row['payprice'], ", bids_won:", bids_won, ", earned_clicks:", earned_clicks, "\n")



    print("__________________________________\n")

    if earned_clicks > 0:
        cpc = total_paid / earned_clicks
    if bids_won > 0:
        ctr = earned_clicks / bids_won

    print("left budget:", budget)
    print("bids_won:", bids_won)
    print("earned clicks:", earned_clicks)
    print("CTR:", ctr)
    print("cost per click:", cpc)




constant_bid: 7 

-- break after auction # 36458
__________________________________

left budget: 0
bids_won: 1239
earned clicks: 0
CTR: 0.0
cost per click: 0


constant_bid: 8 

-- break after auction # 30075
__________________________________

left budget: 0
bids_won: 1164
earned clicks: 0
CTR: 0.0
cost per click: 0


constant_bid: 9 

-- break after auction # 24539
__________________________________

left budget: 0
bids_won: 1084
earned clicks: 1
CTR: 0.0009225092250922509
cost per click: 6250.0


constant_bid: 10 

-- break after auction # 18672
__________________________________

left budget: 0
bids_won: 977
earned clicks: 0
CTR: 0.0
cost per click: 0


constant_bid: 11 

-- break after auction # 14696
__________________________________

left budget: 0
bids_won: 921
earned clicks: 0
CTR: 0.0
cost per click: 0


constant_bid: 12 

-- break after auction # 12445
__________________________________

left budget: 0
bids_won: 835
earned clicks: 1
CTR: 0.0011976047904191617
cost per cl